In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

# examples = [
#     {
#         "movie": "Top Gun",
#         "answer": "🛩️👨‍✈️🔥",
#     },
#     {
#         "movie": "The Godfather",
#         "answer": "👨‍👨‍👦🔫🍝",
#     },
#     {
#         "movie": "@What was the first movie that I asked you about?",
#         "answer": "Your first asked movie is 'Inception'.",
#     },
# ]

# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "{movie}"),
#     ("ai", "{answer}")
# ])

# example_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
# )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        # ("system", "You are a movie to emoji converter. you get movie name, you answer with the 3 emojis that represent the movie."),
        ("system", "You are a movie to emoji converter. you get movie name, you answer with the 3 emojis that represent the movie. But you also get inputs like question. typically questions starts with '@'. if you got question. you answer the question like normal ai chat bot."),
        # example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("The Matrix")

content='🕶️💊🤖'


In [3]:
invoke_chain("The dark knight")

content='🦇🃏🦹\u200d♂️'


In [4]:
invoke_chain("@What was the first movie that I ask you about?")

content="Your first asked movie is 'Top Gun'."
